In [1]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'C:/Users/HP/Desktop/train_test_split/rose/images_rose/images_rose/train'
test_dir = 'C:/Users/HP/Desktop/train_test_split/rose/images_rose/images_rose/test'
validation_dir = 'C:/Users/HP/Desktop/train_test_split/rose/images_rose/images_rose/val'

# Data augmentation for training data
train_datagen = ImageDataGenerator(rescale=1./255)

# Only rescaling for validation and test data
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # DenseNet121 expects 224x224 images
    batch_size=32,
    class_mode='binary'  # Binary classification
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Do not shuffle for evaluation
)

# Load DenseNet121 without the top layer
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers (optional for fine-tuning)
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling
x = Dense(1024, activation='relu')(x)  # Fully connected layer
predictions = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification

# Define the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')



Found 2054 images belonging to 2 classes.
Found 441 images belonging to 2 classes.
Found 441 images belonging to 2 classes.


C:\Users\HP\Documents\anaconda2\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 155s 2s/step - accuracy: 0.8127 - loss: 0.4723 - val_accuracy: 0.9447 - val_loss: 0.2146
Epoch 2/20
 1/64 ━━━━━━━━━━━━━━━━━━━━ 1:26 1s/step - accuracy: 0.9062 - loss: 0.2822

C:\Users\HP\Documents\anaconda2\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


64/64 ━━━━━━━━━━━━━━━━━━━━ 19s 288ms/step - accuracy: 0.9062 - loss: 0.2822 - val_accuracy: 0.9495 - val_loss: 0.2050
Epoch 3/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 107s 2s/step - accuracy: 0.9677 - loss: 0.1643 - val_accuracy: 0.9856 - val_loss: 0.1157
Epoch 4/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 20s 295ms/step - accuracy: 1.0000 - loss: 0.0821 - val_accuracy: 0.9808 - val_loss: 0.1203
Epoch 5/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - accuracy: 0.9919 - loss: 0.0959 - val_accuracy: 0.9904 - val_loss: 0.0865
Epoch 6/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 20s 292ms/step - accuracy: 1.0000 - loss: 0.0596 - val_accuracy: 0.9856 - val_loss: 0.0891
Epoch 7/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - accuracy: 0.9827 - loss: 0.0791 - val_accuracy: 0.9904 - val_loss: 0.0694
Epoch 8/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 19s 282ms/step - accuracy: 1.0000 - loss: 0.0443 - val_accuracy: 0.9904 - val_loss: 0.0686
Epoch 9/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - accuracy: 0.9956 - loss: 0.0546 - val_accuracy: 0.9808 - val_